In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [5]:
data = spark.read.csv('data/SMSSpamCollection1.csv', inferSchema=True, header=True)

In [6]:
data.show()

+--------------------+--------------------+------+--------------------+----+----+----+----+----+----+----+----+----+----+
|                 _c0|                 _c1|   _c2|                 _c3| _c4| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|
+--------------------+--------------------+------+--------------------+----+----+----+----+----+----+----+----+----+----+
|ham\tGo until jur...| crazy.. Availabl...|  null|                null|null|null|null|null|null|null|null|null|null|null|
|ham\tOk lar... Jo...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|
|spam\tFree entry ...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|
|ham\tU dun say so...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|
|ham\tNah I don't ...| he lives around ...|  null|                null|null|null|null|null|null|null|null|null|null|null|
|spam\tFreeMsg Hey...|  

In [7]:
data =data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [8]:
data.show()

+--------------------+--------------------+------+--------------------+----+----+----+----+----+----+----+----+----+----+
|               class|                text|   _c2|                 _c3| _c4| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|
+--------------------+--------------------+------+--------------------+----+----+----+----+----+----+----+----+----+----+
|ham\tGo until jur...| crazy.. Availabl...|  null|                null|null|null|null|null|null|null|null|null|null|null|
|ham\tOk lar... Jo...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|
|spam\tFree entry ...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|
|ham\tU dun say so...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|
|ham\tNah I don't ...| he lives around ...|  null|                null|null|null|null|null|null|null|null|null|null|null|
|spam\tFreeMsg Hey...|  

In [9]:
from pyspark.sql.functions import length

In [10]:
data =data.withColumn('length', length(data['text']))

In [23]:
data.show()

+--------------------+--------------------+------+--------------------+----+----+----+----+----+----+----+----+----+----+------+
|               class|                text|   _c2|                 _c3| _c4| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|length|
+--------------------+--------------------+------+--------------------+----+----+----+----+----+----+----+----+----+----+------+
|ham\tGo until jur...| crazy.. Availabl...|  null|                null|null|null|null|null|null|null|null|null|null|null|    89|
|ham\tOk lar... Jo...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|  null|
|spam\tFree entry ...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|  null|
|ham\tU dun say so...|                null|  null|                null|null|null|null|null|null|null|null|null|null|null|  null|
|ham\tNah I don't ...| he lives around ...|  null|                null|null|null|null|null|null|n

In [12]:
data.groupBy('class').mean().show()

+--------------------+-----------+
|               class|avg(length)|
+--------------------+-----------+
|spam\tfrom www.Ap...|       null|
|spam\t<Forwarded ...|       null|
|ham\tSorry to tro...|        4.0|
|ham\tI love to gi...|       null|
|ham\tThursday nig...|       11.0|
|ham\tI'm at work....|       null|
|ham\tI don't thin...|       null|
|spam\t1000's of g...|       null|
|ham\tAll will com...|       null|
|ham\tBut if she.s...|       null|
|ham\tNt yet chikk...|       null|
|ham\tHello- thanx...|       null|
|ham\tHey hun-onbu...|       null|
|ham\tYou stayin o...|       null|
|ham\tOn the way t...|       null|
|spam\tHope you en...|       null|
|ham\tWhat u talki...|       null|
|ham\tSo i asked h...|       null|
|spam\tYou won't b...|       null|
|ham\tOk thanx... ...|       null|
+--------------------+-----------+
only showing top 20 rows



In [13]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover, CountVectorizer, IDF, StringIndexer)

In [14]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [16]:
from pyspark.ml.feature import VectorAssembler

In [17]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [18]:
from pyspark.ml.classification import NaiveBayes

In [19]:
nb = NaiveBayes()

In [25]:
from pyspark.ml import Pipeline

In [28]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [29]:
cleaner = data_prep_pipe.fit(data)

Py4JJavaError: An error occurred while calling o54.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 23) (DESKTOP-4RQNTUS executor driver): org.apache.spark.SparkException: Failed to execute user defined function (Tokenizer$$Lambda$3452/298675171: (string) => array<string>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at scala.collection.Iterator$$anon$10.h